In [ ]:
import ee
#import ipyleaflet
import geemap

import numpy as np
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay

# replace this with your own project name
ee.Initialize(project='ee-experiments')

In [ ]:
# we will use CDL as training and validation data
# https://www.nass.usda.gov/Research_and_Science/Cropland/SARS1a.php
CDL = ee.ImageCollection('USDA/NASS/CDL').select('cropland').filterDate('2022-01-01', '2022-12-30').first()

USA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties")

# A corn/soy county in Iowa
AOI = USA_COUNTIES.filter(ee.Filter.eq('GEOID', '19073'))

# Approximate corn and soybeans plant/harvest dates in the US
# https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/metadata_ia22.htm
PLANT = '2020-05-15'
HARVEST = '2020-10-15'

# name of the target variable (the label in supervised classification) 
TARGET = 'crop_type'

In [ ]:
def sld_cdl_ramp():
    '''
    Creates an xml Styled Layer Descriptor (sld) from the CDL color palette 
    '''
    cdl_info = CDL.getInfo()
    sld_ramp ='<RasterSymbolizer><ColorMap type="values" extended="true" >'
    for color, crop_id, crop_name in zip(
        cdl_info['properties']['cropland_class_palette'],
        cdl_info['properties']['cropland_class_values'],
        cdl_info['properties']['cropland_class_names']
    ):
        sld_ramp += f'<ColorMapEntry color="#{color}" quantity="{crop_id}" label="{crop_name}" />'
    sld_ramp += '</ColorMap></RasterSymbolizer>'
    return sld_ramp

def sld_three_class_cdl_ramp():
    '''
    Creates a three class xml Styled Layer descriptor (sld) using CDL's
    color palette for corn, soybeans and light blue for other classes
    '''
    sld_ramp ='<RasterSymbolizer><ColorMap type="values" extended="true" >'
    for color, crop_id, crop_name in zip(
        ('#ffd300','#267000','#00a8e2'),
        (1, 2, 3),
        ('corn', 'soybeans', 'other')
    ):
        sld_ramp += f'<ColorMapEntry color="{color}" quantity="{crop_id}" label="{crop_name}" />'

    sld_ramp += '</ColorMap></RasterSymbolizer>'
    return sld_ramp

def maskS2clouds(image):
    '''
    Function to mask clouds using the Sentinel-2 QA band.
    this is copied from 
    https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
    ''' 
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

def mosaic(coll):
    '''
    Mosaic the collection coll
    '''
    unique_dates = coll.aggregate_array('system:time_start').map(
        lambda ts: ee.Date.fromYMD(
            ee.Date(ts).get('year'),
            ee.Date(ts).get('month'),
            ee.Date(ts).get('day')
        )
    ).distinct()
    # print('before mosaicing');
    # print(coll.first().projection());
    mosaic_coll = ee.ImageCollection(
        unique_dates.map(
            lambda d: coll.filterDate(
                ee.Date(d),
                ee.Date(d).advance(1, 'day')
            ).mosaic(
            ).set(
                'system:time_start',
                ee.Date(d)
            )
        )
    )
    # print('after mosaicing')
    # print(mosaic_coll.first().projection())
    return mosaic_coll

def weekly(coll):
    '''
    Creates a weekly mean composites from the input collection coll
    About 20 weeks from Plant to Harvest date, hard coded atm, 
    to be updated later
    '''
    start_date = ee.Date(coll.first().get('system:time_start'))
    end_date = ee.Image(coll.toList(100).get(-1)).get('system:time_start')
    # print(start_date, end_date)
    # print(ee.Date(end_date).difference(ee.Date(start_date), 'week'))
    # print(coll.first().projection())
    # approx 20 weeks
    d_list = ee.List.sequence(1, 20).map(
        lambda week: 
        ee.Feature(
            None, 
            {'s': start_date.advance(ee.Number(week).subtract(1), 'week'),
             'e': start_date.advance(ee.Number(week), 'week')
            }
        )
    )
    # print(d_list);
    weekly_coll = ee.ImageCollection(
        d_list.map(
            lambda date:
            coll.filterDate(
                ee.Date(ee.Feature(date).get('s')),
                ee.Date(ee.Feature(date).get('e'))
            ).mean(
            ).set(
                'system:time_start',
                ee.Date(ee.Feature(date).get('s'))
            )
        )
    ).toBands(
    )
    return weekly_coll

def create_train_test_set(image, sample_size=300, train_size=0.8):
    '''
    Creates a balanced stratified sample of of corn, soy and "other" crops 
    from the input image of size sample_size (each class). The input image 
    must have a TARGET band which is assumed be the reclassified cdl (classes 1, 2 and 3).
    Returns a train and test dataset using train_size proportion
    '''
    features = image.stratifiedSample(**{
      'numPoints': sample_size,
      'classBand': TARGET,
      'region': AOI,
      'scale': 30,
      'geometries': True
    })
    
    features = features.randomColumn()
    train = features.filter(ee.Filter.lte('random', train_size))
    test = features.filter(ee.Filter.gt('random', train_size))
    return train, test

In [ ]:
# remap cdl into corn 1, soy 2, and everything else to 3
CDL_REMAP = CDL.remap([1, 5], [1, 2], 3).rename(TARGET);
# print(cdl_remap);

# make sure it worked
Map = geemap.Map()
Map.center_object(AOI, 10)
Map.addLayer(AOI, {}, 'Greene County IA')
Map.addLayer(ee.Image(CDL).sldStyle(sld_cdl_ramp()).clip(AOI), {}, 'CDL orig')
Map.addLayer(CDL_REMAP.sldStyle(sld_three_class_cdl_ramp()).clip(AOI), {}, 'CDL Remap')
Map

In [ ]:
def train_classifier(train, features, cls=None):
    '''
    Trains a classifier using the training data provided in train.
    cls defaults to random forest if not provided.
    features default to all bands of the training data if not provided.
    '''
    if cls is None:
        # cls = ee.Classifier.smileKNN(**{
        #     'k':1,
        #     'searchMethod':'AUTO',
        #     'metric':'EUCLIDEAN'
        # })
        
        cls = ee.Classifier.smileRandomForest(**{
           'numberOfTrees':30,
           'minLeafPopulation':10
        })

    cls = cls.train(**{
        'features': train,
        'classProperty': TARGET,
        'inputProperties': features
    })
    return cls

def confusion_matrix(test, cls, verbose=False):
    '''
    Returns the confusion matrix of classifier cls applied to test set
    '''
    if verbose:
        cls_info = cls.explain().getInfo()
        del cls_info['trees']
        print(cls_info)

    test = test.classify(cls, 'pred')
    em = test.errorMatrix(TARGET, 'pred', [1, 2, 3])
    # em_info = em.getInfo()
    # print(em.accuracy().getInfo())
    return em

def classification_report(cm):
    '''
    Calculates the common validation metrics from the input confusion matrix cm
    and returns the result as pandas data frame.
    sklearn has a similar function, but the function's protocol doesn't apply here.
    '''
    return pd.DataFrame(
        np.array([
            cm.order().getInfo(), 
            np.ravel(cm.producersAccuracy().getInfo()),
            np.ravel(cm.consumersAccuracy().getInfo()),
            np.ravel(cm.fscore().getInfo()),
            [cm.accuracy().getInfo()] * 3,
            [cm.kappa().getInfo()] * 3
        ]).T,
        columns=['crop_id', 'producersAccuracy', 'consumersAccuracy',
                 'fscore', 'accuracy', 'kappa']
    )

def rf_feature_importance_plot(info):
    importance = pd.Series(info['importance']).sort_values(ascending=True)
    print(importance)
    ax = importance.plot.bar()
    ax.set_title("Random Forest Feature Importances")
    ax.figure.tight_layout()


def main():
    '''Bringing it all together'''

    s2 = ee.ImageCollection('COPERNICUS/S2').filterDate(PLANT, HARVEST).filterBounds(
        AOI
        # ).filter(
        #     ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)
        # ).map(
        #     maskS2clouds
    ).select(
        ['B4', 'B8']
    ).map(
        lambda img: img.divide(10000).copyProperties(img, ["system:time_start"])
        # ).map(
        #     lambda img: img.addBands(
        #         ee.Image.constant(
        #             img.get('system:time_start')
        #         ).rename('timestamp')
        #     )
    )

    img = weekly(mosaic(s2))
    img = img.addBands(CDL_REMAP)
    for sample_size in (300, ):#900, 1200, 1500, 1800, 2100):
        print(f'Sample size:{sample_size}')
        train, test = create_train_test_set(img, sample_size=sample_size, train_size=0.8)
        cls = train_classifier(train=train, features=img.bandNames())
        cm = confusion_matrix(test, cls, verbose=False)
        print(classification_report(cm))
        disp = ConfusionMatrixDisplay(np.array(cm.getInfo()), display_labels=['corn', 'soy', 'other'])
        disp.plot()   
        # rf_feature_importance_plot(cls_info)

    # run inference on the full image
    pred = img.classify(cls)
    return pred

pred = main()

In [ ]:
Map = geemap.Map()
Map.center_object(AOI, 10)
Map.addLayer(AOI, {}, 'Greene County IA')
Map.addLayer(CDL_REMAP.sldStyle(sld_three_class_cdl_ramp()).clip(AOI), {}, 'true labels')
Map.addLayer(pred.sldStyle(sld_three_class_cdl_ramp()).clip(AOI), {}, 'predicted labels')
Map